In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

data = keras.datasets.imdb

In [ ]:
(train_data, train_labels), (test_data, test_labels) = data.load_data(num_words=88000)

17465344/17464789 [==============================] - 0s 0us/step


In [ ]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [ ]:
print(train_labels)

[1 0 0 ... 0 1 0]


In [ ]:
word_index = data.get_word_index()

1646592/1641221 [==============================] - 0s 0us/step


In [ ]:
word_index = {k:(v + 3) for k, v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [ ]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value = word_index["<PAD>"], padding= "post", maxlen= 250)

In [ ]:
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value = word_index["<PAD>"], padding= "post", maxlen= 250)

In [ ]:
len(train_data[0]), len(test_data[45])

(250, 250)

In [ ]:
def decode_review(text):
  return " ".join([reverse_word_index.get(i, "?") for i in text])

In [ ]:
decode_review(test_data[2])

"king irritated jupiter sends them a stork br br delighted with this formidable looking new king who towers above them the frogs welcome him with a delegation of formally dressed dignitaries the mayor steps forward to hand him the key to the commonwealth as newsreel cameras record the event to everyone's horror the stork promptly eats the mayor and then goes on a merry rampage swallowing citizens at random a title card dryly reads news of the king's appetite throughout the kingdom when the now terrified frogs once more beseech jupiter for help he loses his temper and showers their community with lightning bolts the moral of our story delivered by a hapless frog just before he is eaten is let well enough alone br br considering the time period when this startling little film was made and considering the fact that it was made by a russian émigré at the height of that beleaguered country's civil war it would be easy to see this as a parable about those events starewicz may or may not have

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Embedding(88000, 16))
model.add(keras.layers.GlobalAveragePooling1D())  #you take the average of this words 
model.add(keras.layers.Dense(16, activation="relu"))
model.add(keras.layers.Dense(1, activation = "sigmoid")) #sigmoid [0-1]

model.summary()

model.compile(optimizer='adam',
              loss = "binary_crossentropy",
              metrics=['accuracy'])

x_val = train_data[:1000]
x_train = train_data[1000:]

y_val = train_labels[:1000]
y_train = train_labels[1000:]

history = model.fit(x_train, y_train, epochs=40, batch_size = 52, validation_data=(x_val, y_val), verbose=1) # what is verbose ?/ what is batch_size ?

results = model.evaluate(test_data, test_labels)

model.save("model.h5")#extension for tensorflow 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          1408000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 1,408,289
Trainable params: 1,408,289
Non-trainable params: 0
_________________________________________________________________
Epoch 1/40
233/462 [==============>...............] - ETA: 3s - loss: 0.6708 - accuracy: 0.6605

KeyboardInterrupt: ignored

In [ ]:
x_train

array([[  85,  857,   13, ...,   30,  184,   52],
       [   1,   13,   69, ...,    0,    0,    0],
       [   1,  684,   91, ...,    0,    0,    0],
       ...,
       [   1,   11,    6, ...,    0,    0,    0],
       [   1, 1446, 7079, ...,    0,    0,    0],
       [   1,   17,    6, ...,    0,    0,    0]], dtype=int32)

In [ ]:
print(results)


[1.3680107593536377, 0.8445600271224976]


In [ ]:
print(model.predict(test_data[0]))

In [ ]:
test_review = test_data[10]
predict = model.predict_classes([test_review])
print("Review: ")
print(decode_review(test_review))
print("Prediction: " + str(predict[0]))
print("Actual: " + str(test_labels[0]))
print(results)

Review: 
<START> inspired by hitchcock's strangers on a train concept of two men swapping murders in exchange for getting rid of the two people messing up their lives throw momma from the train is an original and very inventive comedy take on the idea it's a credit to danny devito that he both wrote and starred in this minor comedy gem br br anne ramsey is the mother who inspires the film's title and it's understandable why she gets under the skin of danny devito with her sharp tongue and relentlessly putting him down for any minor infraction billy crystal is the writer who's wife has stolen his book idea and is now being lionized as a great new author even appearing on the oprah show to in adulation he should be enjoying thus devito gets the idea of swapping murders to rid themselves of these nuisance factors br br of course everything and anything can happen when writer carl reiner lets his imagination roam with unending ideas for how the plot develops and it's amusing all the way th